written by Hamid Nemati 9535023

# Forward-Backward

In [1]:
import numpy as np
import matplotlib.pyplot as P

In [2]:
N=3 # size of states space
M=4 # size of oservations space
T=5 # number of times we see an observation

In [3]:
# generate a random sequence of observations
# numpy.random.randint(low, high=None, size=None, dtype=int)
O = np.random.randint(0,M,T)
print(O)

[0 3 3 3 0]


In [4]:
# observation matrix
b = np.random.random((N,M))
b /= np.array([b.sum(axis=0)]) 
print(b)

[[0.29422203 0.07484759 0.03951524 0.13112228]
 [0.44855994 0.39854757 0.34780568 0.39427893]
 [0.25721804 0.52660484 0.61267908 0.47459878]]


In [5]:
# probabilites for individual events have to sum to 1
print(b.sum(axis=0))

[1. 1. 1. 1.]


In [6]:
# Given thes probabilities, we can easily estimate the observation probabilities in time.
# we have 5 observation so we have 5 rows.
# we have 3 states so we have 3 columns.
ob=b.T[O]
print(ob)

[[0.29422203 0.44855994 0.25721804]
 [0.13112228 0.39427893 0.47459878]
 [0.13112228 0.39427893 0.47459878]
 [0.13112228 0.39427893 0.47459878]
 [0.29422203 0.44855994 0.25721804]]


In [7]:
#state transition matrix
a = np.random.random((N,N))
a /= np.array([a.sum(axis=-1)]).T
print(a)
print(a.sum(axis=1))

[[0.24516257 0.42538428 0.32945316]
 [0.1551306  0.49998497 0.34488443]
 [0.31759074 0.28453531 0.39787395]]
[1. 1. 1.]


In [8]:
# generate the priors
pi = np.random.random(N)
pi /= pi.sum()
print(pi)

[0.56694725 0.27998804 0.15306471]


In [9]:
hmm =(O,pi,a,b,N,M,T)

In [10]:
def forward(O,pi,a,b,N,M,T):
    fwd = np.zeros((T,N))

    #initialization
    fwd[0] = pi*b[:,O[0]]

    #induction:
    for t in range(T-1):
        fwd[t+1] = np.dot(fwd[t],a)*b[:,O[t+1]]
    
    return fwd

print(forward(*hmm))

[[0.16680837 0.12559142 0.03937101]
 [0.00955647 0.05715228 0.0540733 ]
 [0.00372153 0.01893572 0.0210597 ]
 [0.0013818  0.00671965 0.00765804]
 [0.00112196 0.0027481  0.00149693]]


In [11]:
# sum of all the forward probabilities at the last time step
def full_prob(fwd):
    return fwd[-1].sum()

print(full_prob(forward(*hmm)))

0.005366989247259792


In [12]:
def backward(O,pi,a,b,N,M,T):
    bk=np.zeros((T,N))
    
    #initialization:
    bk[T-1] = 1
    
    #induction:
    for t in reversed(range(T-1)):
        bk[t] = np.dot(bk[t+1]*b[:,O[t+1]],a.T)
    return bk

print(backward(*hmm))

[[0.01583352 0.01700768 0.01498074]
 [0.04394399 0.04719749 0.04160271]
 [0.12189395 0.1307065  0.11578202]
 [0.34768387 0.35862656 0.32341369]
 [1.         1.         1.        ]]


When we have both Forward and Backward probabilities computed, we can use them together to estimate the probability of each state, at each point in time, by simply multuplying those two probablilities together. Addtionally, we normalize this by the joint probability $P$ to make sure everything sums up nicely (i.e. each timestep sums to 1):

In [13]:
def gamma(fwd,bk,fp):
    return (fwd*bk)/fp

print(gamma(forward(*hmm),backward(*hmm),full_prob(forward(*hmm))))

[[0.49211274 0.39799197 0.10989528]
 [0.07824673 0.50259912 0.41915415]
 [0.08452263 0.46115656 0.45432082]
 [0.08951567 0.44901254 0.46147179]
 [0.20904832 0.51203824 0.27891344]]


# Viterbi

In [14]:
# d(t, i): path with the highest probability of seeing the first t observation then ending in State i
# ph(t, i): if we are in state i in time t, what is the privious state?
def viterbi(O,pi,a,b,N,M,T):
    d = np.zeros((T,N))
    ph = np.zeros((T,N), dtype=np.int)

    #initialization
    d[0] = pi*b[:,O[0]]
    ph[0] = 0

    #recursion
    for t in range(1,T):
        m = d[t-1] * a.T        
        ph[t] = m.argmax(axis=1)
        d[t] = m[np.arange(N), ph[t]] * b[:,O[t]]    

    #termination
    Q = np.zeros(T,dtype=np.int)
    Q[T-1] = np.argmax(d[T-1])
    Pv = d[T-1, Q[T-1]]

    #path back-tracking
    for t in reversed(range(T-1)):
        Q[t] = ph[t+1, Q[t+1]]

    return Q

print(viterbi(*hmm))

[0 1 1 1 1]


In [15]:
print('Best observation sequence: {}'.format(np.argmax(ob,axis=1)))
print('Best forward prob. sequence: {}'.format(np.argmax(forward(*hmm),axis=1)))
print('Best backward prob. sequence: {}'.format(np.argmax(backward(*hmm),axis=1)))
g = gamma(forward(*hmm),backward(*hmm),full_prob(forward(*hmm)))
print('Best full prob. sequence: {}'.format(np.argmax(g,axis=1)))
print('Best Viterbi sequence: {}'.format(viterbi(*hmm)))

Best observation sequence: [1 2 2 2 1]
Best forward prob. sequence: [0 1 2 2 1]
Best backward prob. sequence: [1 1 1 1 0]
Best full prob. sequence: [0 1 1 2 1]
Best Viterbi sequence: [0 1 1 1 1]


# Baum-Welch

Now we can also determine the sequence probablity, which would help us with fine-tuning the state transition likelihood.

The transition probability computed for each timestep gives us a 3D matrix of size 4x3x3, since there are $N^2$ transitions at each timestep and only $T-1$ transitions time steps (i.e. number of transitions between 5 consecutive timesteps).

In [16]:
# The transition probability computed for each timestep
def xi(fwd,bk,fp,O,pi,a,b,N,M,T):
    return  fwd[:-1].reshape((T-1,N,1))*\
            a.reshape((1,N,N))*\
            b[:,O[1:]].T.reshape((T-1,1,N))*\
            bk[1:].reshape((T-1,1,N))/fp

print(xi(forward(*hmm),backward(*hmm),full_prob(forward(*hmm)),*hmm))

[[[0.04390533 0.24603168 0.20217573]
  [0.02091716 0.21772519 0.15934963]
  [0.01342424 0.03884225 0.05762879]]

 [[0.00697717 0.03903452 0.03223503]
  [0.02640334 0.2743851  0.20181068]
  [0.05114211 0.14773693 0.2202751 ]]

 [[0.00775008 0.0417079  0.03506465]
  [0.02495225 0.24943315 0.18677115]
  [0.05681335 0.15787149 0.23963598]]

 [[0.01857137 0.04912657 0.02181773]
  [0.05714637 0.28079764 0.11106853]
  [0.13333058 0.18211403 0.14602718]]]


In [17]:
# The expected prior value can be estimated by simply looking at the 
# expected value (gamma) at the first point in time
def exp_pi(gamma):
    return gamma[0]

print(exp_pi(gamma(forward(*hmm),backward(*hmm),full_prob(forward(*hmm)))))

[0.49211274 0.39799197 0.10989528]


In [18]:
# The expected transistions is the sum (for different timestep) of transition 
# probabilities xi normalized by the corresponding emitting state probabilities
def exp_a(gamma,xi,N):            
    return xi[:].sum(axis=0)/gamma[:-1].sum(axis=0).reshape(N,1)

fw = forward(*hmm)
bk = backward(*hmm)
fp = full_prob(fw)
g  = gamma(fw,bk,fp)
x  = xi(fw,bk,fp,*hmm)

print(exp_a(g,x,N))

[[0.1037133  0.50497286 0.39131383]
 [0.07147226 0.5645922  0.36393554]
 [0.17628936 0.36444448 0.45926616]]


In [19]:
# the expected observation likelihood is sum of probabilities of given states at times where
# the specific observation occured normalized by the sum of all the probilities in time 
# for the given state
def exp_b(gamma,O,N,M):    
    return np.array(list(map(lambda k: np.sum(gamma[O==k],axis=0)/np.sum(gamma,axis=0), np.arange(M)))).T

fw = forward(*hmm)
bk = backward(*hmm)
fp = full_prob(fw)
g  = gamma(fw,bk,fp)

print(exp_b(g,O,N,M))

[[0.73539665 0.         0.         0.26460335]
 [0.39178183 0.         0.         0.60821817]
 [0.22555909 0.         0.         0.77444091]]


The Baum-Welch algorithm now works by repeadatly updating the model parameters in this two step EM (first Expectation, then Modification) procedure. The goal of the algorithm is to maximize the $P(O|\Lambda)$, where $O$ is the observation sequence and $\Lambda$ is the model. We will also print out the mean square of the difference of model parameters vs their expected value:

In [20]:
print('Initial probability: {}'.format(full_prob(forward(*hmm))))

hmm_new=hmm
for i in range(15):
    fw = forward(*hmm_new)
    bk = backward(*hmm_new)
    fp = full_prob(fw)
    g  = gamma(fw,bk,fp)
    x  = xi(fw,bk,fp,*hmm_new)

    pi_new = exp_pi(g)
    a_new  = exp_a(g,x,N)
    b_new  = exp_b(g,O,N,M)
    
    err = np.concatenate(((pi_new-hmm_new[1]).ravel(),(a_new-hmm_new[2]).ravel(),(b_new-hmm_new[3]).ravel()))    

    hmm_new = (O,pi_new,a_new,b_new,N,M,T)

    print('Update #{} probability: {} -- mean error: {}'.format(i+1,full_prob(forward(*hmm_new)),np.mean(err**2)))

Initial probability: 0.005366989247259792
Update #1 probability: 0.051064857052248995 -- mean error: 0.05771785410418955
Update #2 probability: 0.07094211999280131 -- mean error: 0.00396883133331852
Update #3 probability: 0.09603175883786529 -- mean error: 0.003350119865485326
Update #4 probability: 0.11277771061930612 -- mean error: 0.0012735524504434638
Update #5 probability: 0.1204017979465175 -- mean error: 0.0003245524207321524
Update #6 probability: 0.1246232203473026 -- mean error: 0.0001649316486886098
Update #7 probability: 0.1280278518201862 -- mean error: 0.00015484004081682884
Update #8 probability: 0.13143492544939248 -- mean error: 0.00017666660529634364
Update #9 probability: 0.13527159603211697 -- mean error: 0.0002279714869739399
Update #10 probability: 0.14019818530610198 -- mean error: 0.0003295356487009543
Update #11 probability: 0.14763507881067328 -- mean error: 0.0005166788857658726
Update #12 probability: 0.16028413376542416 -- mean error: 0.0008040879841357417
